In [1]:
#pip installs:

#pip install lumibot timedelta alpaca-trade-api
# side note for gym: gym-trading-env 0.3.3 requires ccxt==3.0.59

In [92]:
#import everything
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
import math
import time
import keyboard
import matplotlib.pyplot as plt
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from transformers import AdamW
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification
import spacy
import pydantic
import requests
from bs4 import BeautifulSoup
from lumibot.brokers import Alpaca
from alpaca_trade_api import REST
from datetime import datetime, timedelta 
import time as Time

In [3]:
# Setup the API
API_KEY = "PKWG828Y4OQ3Q081LAH0" 
API_SECRET = "tS8dvpp3GEXUdOfg5Nbtjn81C0y5Lghpidxr8Guy" 
BASE_URL = "https://paper-api.alpaca.markets/v2"
ALPACA_CREDS = {
    "API_KEY":API_KEY, 
    "API_SECRET": API_SECRET, 
    "PAPER": True
}
api = REST(base_url=BASE_URL, key_id=API_KEY, secret_key=API_SECRET)

In [4]:
help(api.get_news)

Help on method get_news in module alpaca_trade_api.rest:

get_news(symbol: Union[str, List[str], NoneType] = None, start: Optional[str] = None, end: Optional[str] = None, limit: int = 10, sort: alpaca_trade_api.rest.Sort = <Sort.Desc: 'desc'>, include_content: bool = False, exclude_contentless: bool = False) -> alpaca_trade_api.entity_v2.NewsListV2 method of alpaca_trade_api.rest.REST instance



In [64]:
def to_strftime(time):
    try:
        time = datetime.strptime(time, "%Y-%m-%d")
        time = time.strftime('%Y-%m-%d')
    except ValueError as e:
        print(f"Error parsing date: {e}")
    return time

In [95]:
years=[]
for i in range(23, -1, -1):
    year = f"20{i}"
    if i < 10:
        year = f"200{i}"
    years.append(year)

dates = []
for i in range(len(years)):
    d1 = years[i] + "-01-01"
    d2 = years[i] + "-12-31"
    d1 = to_strftime(d1)
    d2 = to_strftime(d2)
    dates.append([d1,d2])

for i in range(len(dates)):
    print("From", dates[i][0], "to", dates[i][1])
    #time.sleep(1)
        
# start = "2024-08-15"
# end = "2000-01-01"

# # Convert date strings to datetime objects
# try:
#     today = datetime.strptime(start, "%Y-%m-%d")
#     prior_date = datetime.strptime(end, "%Y-%m-%d")
#     print("From", today, "to", prior_date)
#     today = today.strftime('%Y-%m-%d')
#     prior_date = prior_date.strftime('%Y-%m-%d')
# except ValueError as e:
#     print(f"Error parsing date: {e}")
# print("From", today, "to", prior_date)






From 2023-01-01 to 2023-12-31
From 2022-01-01 to 2022-12-31
From 2021-01-01 to 2021-12-31
From 2020-01-01 to 2020-12-31
From 2019-01-01 to 2019-12-31
From 2018-01-01 to 2018-12-31
From 2017-01-01 to 2017-12-31
From 2016-01-01 to 2016-12-31
From 2015-01-01 to 2015-12-31
From 2014-01-01 to 2014-12-31
From 2013-01-01 to 2013-12-31
From 2012-01-01 to 2012-12-31
From 2011-01-01 to 2011-12-31
From 2010-01-01 to 2010-12-31
From 2009-01-01 to 2009-12-31
From 2008-01-01 to 2008-12-31
From 2007-01-01 to 2007-12-31
From 2006-01-01 to 2006-12-31
From 2005-01-01 to 2005-12-31
From 2004-01-01 to 2004-12-31
From 2003-01-01 to 2003-12-31
From 2002-01-01 to 2002-12-31
From 2001-01-01 to 2001-12-31
From 2000-01-01 to 2000-12-31


In [97]:
# numpy vectorize function for converting np array to strftime
vectorize_to_strftime  = np.vectorize(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%SZ").strftime("%Y-%m-%d"))
# Stock
token = "SPY"
data = pd.DataFrame(columns=["text","date"])

for i in range(len(dates)):
    start_ = dates[i][0]
    end_ = dates[i][1]
    print("From", start_, "to", end_)
    news = api.get_news(token, start=start_, end=end_, limit=20000)# , exclude_contentless=True makes it so news with no summary are not recieved
    print("lenghts is: ", len(news))

    if len(news) == 0:
        print("Lenght is 0, going next year")
        continue
    
    print("Succesfully appended data")
    
    headlines = [ev.__dict__["_raw"]["headline"] for ev in news]
    summaries = [ev.__dict__["_raw"]["summary"] for ev in news]
    text = np.char.add(np.char.add(headlines, '. '), summaries)
    
    time = [ev.__dict__["_raw"]["created_at"] for ev in news]
    time = vectorize_to_strftime(time)
    
    df = pd.DataFrame({
        'text': text,
        'date': time
    })
    #df = pd.DataFrame(combined_array, columns=['news'])
    data = pd.concat([data, df], axis=0)
    print("Sleeping for 30sec\n")
    Time.sleep(30)
data = data.reset_index(drop=True)
data

From 2023-01-01 to 2023-12-31
lenghts is:  8959
Succesfully appended data
Sleeping for 30sec

From 2022-01-01 to 2022-12-31
lenghts is:  8487
Succesfully appended data
Sleeping for 30sec

From 2021-01-01 to 2021-12-31
lenghts is:  8312
Succesfully appended data
Sleeping for 30sec

From 2020-01-01 to 2020-12-31
lenghts is:  13276
Succesfully appended data
Sleeping for 30sec

From 2019-01-01 to 2019-12-31
lenghts is:  8487
Succesfully appended data
Sleeping for 30sec

From 2018-01-01 to 2018-12-31
lenghts is:  4603
Succesfully appended data
Sleeping for 30sec

From 2017-01-01 to 2017-12-31
lenghts is:  2507
Succesfully appended data
Sleeping for 30sec

From 2016-01-01 to 2016-12-31
lenghts is:  2336
Succesfully appended data
Sleeping for 30sec

From 2015-01-01 to 2015-12-31
lenghts is:  1417
Succesfully appended data
Sleeping for 30sec

From 2014-01-01 to 2014-12-31
lenghts is:  0
Lenght is 0, going next year
From 2013-01-01 to 2013-12-31
lenghts is:  0
Lenght is 0, going next year
From 

,text,date
0,Tom Lee Predicts S&P 500 Surge In 2024 Despite...,2023-12-30
1,"2023's Top Dogs: Nvidia, Meta Lead S&P 500 Cha...",2023-12-29
2,"AI Revolution, Fed's Pivot Power US Stock Mark...",2023-12-29
3,Mickey Mouse Goes Public: Disney Icon Enters D...,2023-12-29
4,"Market Clubhouse Morning Memo - December 29th,...",2023-12-29
...,...,...
58379,Wainscott Capital Founder Fatally Shot By Son.,2015-01-05
58380,10 Of The Most Popular Stocks In 2014.,2015-01-03
58381,Senate Majority Leader Reid Suffers Broken Rib...,2015-01-02
58382,Cleveland Fed Pres Mester Recently Speaking on...,2015-01-02


In [107]:
data.to_csv('news_data.csv', index=True)

In [108]:
data_back = pd.read_csv('news_data.csv')
data_back

,Unnamed: 0,text,date
0,0,Tom Lee Predicts S&P 500 Surge In 2024 Despite...,2023-12-30
1,1,"2023's Top Dogs: Nvidia, Meta Lead S&P 500 Cha...",2023-12-29
2,2,"AI Revolution, Fed's Pivot Power US Stock Mark...",2023-12-29
3,3,Mickey Mouse Goes Public: Disney Icon Enters D...,2023-12-29
4,4,"Market Clubhouse Morning Memo - December 29th,...",2023-12-29
...,...,...,...
58379,58379,Wainscott Capital Founder Fatally Shot By Son.,2015-01-05
58380,58380,10 Of The Most Popular Stocks In 2014.,2015-01-03
58381,58381,Senate Majority Leader Reid Suffers Broken Rib...,2015-01-02
58382,58382,Cleveland Fed Pres Mester Recently Speaking on...,2015-01-02
